# WIP

```{cypher}
CALL gds.graph.project(
  'testPRGraph',
  {
    Artist: {},
    Tag: {},
    Track: {}
  },
  {
    COLLAB_WITH: { properties: 'pg_weight' },
    HAS_TAG: { properties: 'pg_weight' },
    LINKED_TO: { properties: 'pg_weight' },
    MUSICALLY_RELATED_TO: { properties: 'pg_weight' },
    PERSONALLY_RELATED_TO: { properties: 'pg_weight' },
    TAGS: { properties: 'pg_weight' },
    WORKED_BY: { properties: 'pg_weight' },
    WORKED_IN: { properties: 'pg_weight' }
  }
);
```

AND

```{cypher}
CALL gds.pageRank.stream(
    'testPRGraph',
    {
        nodeLabels: ['Artist', 'Tag', 'Track'],
        relationshipTypes: ['COLLAB_WITH', 'HAS_TAG', 'LINKED_TO', 'MUSICALLY_RELATED_TO', 'PERSONALLY_RELATED_TO', 'TAGS', 'WORKED_BY', 'WORKED_IN'],
        concurrency: 4,
        logProgress: true,
        dampingFactor: 0.85,
        maxIterations: 20,
        tolerance: 0.0000000001,
        relationshipWeightProperty: 'pg_weight'
    }
);
```

is too slow.

TODO: Try with nx

In [49]:
from typing import Any
from neo4j import GraphDatabase, basic_auth, Driver
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import os
from sklearn.preprocessing import MinMaxScaler
import networkx as nx
from scipy.stats import pearsonr

In [50]:
load_dotenv()

True

In [51]:
DB_HOST = os.getenv("NEO4J_HOST")
DB_PORT = os.getenv("NEO4J_PORT")
DB_USER = os.getenv("NEO4J_USER")
DB_PASS = os.getenv("NEO4J_PASS")

# .env validation
assert DB_HOST is not None and \
    DB_PORT is not None and \
    DB_USER is not None and \
    DB_PASS is not None, \
    "INVALID .env"

In [52]:
driver = GraphDatabase.driver(f"bolt://{DB_HOST}:{DB_PORT}", auth=basic_auth(DB_USER, DB_PASS))
driver

In [53]:
def execute_query_return(driver: Driver, query: str, params: None | dict[str, Any] = None) -> list[dict[str, Any]]:
    with driver.session() as session:
        result = session.run(query, params)  # type: ignore
        return result.data()

In [54]:
query = """
match (n:Artist {in_last_fm: true})-[:COLLAB_WITH]->()
with distinct(n) as n
return n.main_id as main_id, n.listeners as listeners, n.playcount as playcount;
"""

data = pd.DataFrame(execute_query_return(driver, query))
len(data)

KeyboardInterrupt: 

In [7]:
data.corr().loc["listeners", "playcount"]

0.6955042178755003

In [8]:
mmscaler = MinMaxScaler()

data["listeners_scaled"] = mmscaler.fit_transform(data[["listeners"]])
data["playcount_scaled"] = mmscaler.fit_transform(data[["playcount"]])
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled
count,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05
mean,2.608033e+04,6.750095e+05,0.002467,2.512589e-04
std,1.667765e+05,1.051326e+07,0.015773,3.913353e-03
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00
25%,3.400000e+01,1.700000e+02,0.000003,6.327912e-08
50%,4.290000e+02,2.696000e+03,0.000041,1.003532e-06
75%,5.140000e+03,3.869600e+04,0.000486,1.440382e-05
max,1.057367e+07,2.686510e+09,1.000000,1.000000e+00


In [9]:
data.head()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled
0,1889201,284,659,2.685916e-05,2.452997e-07
1,1037506,4,5,3.782980e-07,1.861151e-09
2,2557269,103,417,9.741174e-06,1.552200e-07
3,1039018,622,3274,5.882534e-05,1.218681e-06
4,569258,1930,5911,1.825288e-04,2.200252e-06


In [10]:
pca_calculator = PCA(1)
pca = pca_calculator.fit_transform(data[["listeners_scaled", "playcount_scaled"]])

In [11]:
pca_calculator.explained_variance_ratio_  # This is great

array([0.97095481])

In [61]:
pca_calculator.components_

(1, 2)

In [62]:
a, b = pca_calculator.components_.flatten()
print(a, b)

0.9845163377263673 0.17529284283695573


In [64]:
pca

array([[-0.0024459 ],
       [-0.00247201],
       [-0.00246277],
       ...,
       [-0.00246074],
       [-0.00247173],
       [-0.00242333]])

In [73]:
pca.flatten() == a * data.listeners_scaled + b * data.playcount_scaled

0         False
1         False
2         False
3         False
4         False
          ...  
512744    False
512745    False
512746    False
512747    False
512748    False
Length: 512749, dtype: bool

In [14]:
data["popularity"] = pca
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity
count,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05,5.127490e+05
mean,2.608033e+04,6.750095e+05,0.002467,2.512589e-04,4.100959e-19
std,1.667765e+05,1.051326e+07,0.015773,3.913353e-03,1.601328e-02
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-2.472388e-03
25%,3.400000e+01,1.700000e+02,0.000003,6.327912e-08,-2.469214e-03
50%,4.290000e+02,2.696000e+03,0.000041,1.003532e-06,-2.432211e-03
75%,5.140000e+03,3.869600e+04,0.000486,1.440382e-05,-1.990640e-03
max,1.057367e+07,2.686510e+09,1.000000,1.000000e+00,1.003843e+00


In [15]:
data["popularity_scaled"] = mmscaler.fit_transform(data[["popularity"]])
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled
count,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05,5.127490e+05,512749.000000
mean,2.608033e+04,6.750095e+05,0.002467,2.512589e-04,4.100959e-19,0.002457
std,1.667765e+05,1.051326e+07,0.015773,3.913353e-03,1.601328e-02,0.015913
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-2.472388e-03,0.000000
25%,3.400000e+01,1.700000e+02,0.000003,6.327912e-08,-2.469214e-03,0.000003
50%,4.290000e+02,2.696000e+03,0.000041,1.003532e-06,-2.432211e-03,0.000040
75%,5.140000e+03,3.869600e+04,0.000486,1.440382e-05,-1.990640e-03,0.000479
max,1.057367e+07,2.686510e+09,1.000000,1.000000e+00,1.003843e+00,1.000000


In [16]:
graph = nx.Graph()

# Nodes
query = "match (n:Artist {in_last_fm: true}) return n.main_id as main_id;"
with driver.session() as session:
    records = session.run(query)
    for i, artist in enumerate(records):
        graph.add_node(artist["main_id"])

        if i % 1000 == 0:
            print(i)

print("NODES DONE")

# Edges

query = """
    match (a:Artist {in_last_fm: true})-[r:COLLAB_WITH]->(b:Artist {in_last_fm: true})
    where a < b
    return a.main_id as a, b.main_id as b, r.count as weight
    ;
"""

with driver.session() as session:
    records = session.run(query)
    for i, rel in enumerate(records):
        a, b, w = rel["a"], rel["b"], rel["weight"]

        graph.add_edge(a, b, weight=float(w))

        if i % 1000 == 0:
            print(i)

print("DONE!")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [17]:
pg = nx.pagerank(graph)
len(pg)

512749

In [18]:
len(data)

512749

In [19]:
pg.get(data.loc[0, "main_id"])

1.53493754810374e-06

In [20]:
data["pagerank"] = data["main_id"].map(pg)
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank
count,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05
mean,2.608033e+04,6.750095e+05,0.002467,2.512589e-04,4.100959e-19,0.002457,1.950272e-06
std,1.667765e+05,1.051326e+07,0.015773,3.913353e-03,1.601328e-02,0.015913,2.920956e-06
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-2.472388e-03,0.000000,3.005595e-07
25%,3.400000e+01,1.700000e+02,0.000003,6.327912e-08,-2.469214e-03,0.000003,8.071427e-07
50%,4.290000e+02,2.696000e+03,0.000041,1.003532e-06,-2.432211e-03,0.000040,1.501316e-06
75%,5.140000e+03,3.869600e+04,0.000486,1.440382e-05,-1.990640e-03,0.000479,1.997536e-06
max,1.057367e+07,2.686510e+09,1.000000,1.000000e+00,1.003843e+00,1.000000,6.076876e-04


In [21]:
data.sort_values(by="pagerank", inplace=True)
data.reset_index(drop=True, inplace=True)
data.head()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank
0,1791908,212,951,2.004980e-05,3.539909e-07,-0.002453,1.967714e-05,3.005595e-07
1,2373904,1,21,9.457451e-08,7.816833e-09,-0.002472,9.388745e-08,3.005595e-07
2,2750702,390,1632,3.688406e-05,6.074796e-07,-0.002436,3.619089e-05,3.005595e-07
3,1551391,24,174,2.269788e-06,6.476805e-08,-0.002470,2.231902e-06,3.005595e-07
4,2458828,139,651,1.314586e-05,2.423218e-07,-0.002459,1.290330e-05,3.005595e-07


In [22]:
data.corr()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank
main_id,1.000000,-0.082811,-0.031470,-0.082811,-0.031470,-0.081653,-0.081653,-0.161269
listeners,-0.082811,1.000000,0.695504,1.000000,0.695504,0.999526,0.999526,0.187253
playcount,-0.031470,0.695504,1.000000,0.695504,1.000000,0.717291,0.717291,0.088277
listeners_scaled,-0.082811,1.000000,0.695504,1.000000,0.695504,0.999526,0.999526,0.187253
playcount_scaled,-0.031470,0.695504,1.000000,0.695504,1.000000,0.717291,0.717291,0.088277
popularity,-0.081653,0.999526,0.717291,0.999526,0.717291,1.000000,1.000000,0.185367
popularity_scaled,-0.081653,0.999526,0.717291,0.999526,0.717291,1.000000,1.000000,0.185367
pagerank,-0.161269,0.187253,0.088277,0.187253,0.088277,0.185367,0.185367,1.000000


In [23]:
data['pagerank_acc'] = data['pagerank'].cumsum()
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
count,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05,5.127490e+05,512749.000000,5.127490e+05,5.127490e+05
mean,2.608033e+04,6.750095e+05,0.002467,2.512589e-04,4.711555e-19,0.002457,1.950272e-06,2.812462e-01
std,1.667765e+05,1.051326e+07,0.015773,3.913353e-03,1.601328e-02,0.015913,2.920956e-06,2.423933e-01
min,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,-2.472388e-03,0.000000,3.005595e-07,3.005595e-07
25%,3.400000e+01,1.700000e+02,0.000003,6.327912e-08,-2.469214e-03,0.000003,8.071427e-07,6.888154e-02
50%,4.290000e+02,2.696000e+03,0.000041,1.003532e-06,-2.432211e-03,0.000040,1.501316e-06,2.141772e-01
75%,5.140000e+03,3.869600e+04,0.000486,1.440382e-05,-1.990640e-03,0.000479,1.997536e-06,4.547600e-01
max,1.057367e+07,2.686510e+09,1.000000,1.000000e+00,1.003843e+00,1.000000,6.076876e-04,1.000000e+00


In [24]:
data.tail()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
512744,255180,2321703,47773305,0.219574,0.017783,0.216819,0.217915,0.000180,0.998608
512745,762646,1403,13224,0.000133,0.000005,-0.002341,0.000131,0.000229,0.998837
512746,177,3979125,68405038,0.376324,0.025462,0.372488,0.372607,0.000267,0.999104
512747,2327,2288231,87452010,0.216408,0.032552,0.216291,0.217391,0.000289,0.999392
512748,491324,75375,8583151,0.007129,0.003195,0.005106,0.007531,0.000608,1.000000


In [25]:
data.pagerank_acc.corr(data.popularity_scaled)

0.10072488637824047

In [26]:
i = 0
current_cum_sum = 0
while i < len(data) - 1:
    if data.loc[i, "pagerank"] != data.loc[i + 1, "pagerank"]:
        current_cum_sum += data.loc[i, "pagerank"]
        data.loc[i, "pagerank_acc"] = current_cum_sum
        i += 1
    else:
        j = i + 1
        next_cum_sum = current_cum_sum + data.loc[i, "pagerank"]
        while data.loc[j, "pagerank"] == data.loc[i, "pagerank"] and  j < len(data) - 1:
            next_cum_sum += data.loc[j, "pagerank"]
            j += 1
        mean_cum_sum = (next_cum_sum + current_cum_sum) / 2.0
        for k in range(i, j):
            data.loc[k, "pagerank_acc"] = mean_cum_sum
        current_cum_sum = next_cum_sum
        i = j
    if i % 1000 == 0:
        print(i, current_cum_sum)
data.loc[i, "pagerank_acc"] = current_cum_sum + data.loc[i, "pagerank"]

17000 0.005187986623952476
18000 0.005533832003239527
19000 0.005886067382409674
20000 0.00624413939152666
22000 0.006974931259962907
23000 0.00734730394421146
24000 0.007724205153150412
26000 0.008491649745773395
27000 0.008881926775497723
28000 0.009276521817525158
29000 0.009675525996783825
31000 0.01048590668443538
32000 0.010897344038008787
33000 0.011312470337214255
34000 0.011731376927247709
35000 0.012154095375156177
37000 0.01301115642591899
39000 0.013883073139963802
41000 0.014770078229272012
42000 0.015219568705988704
45000 0.016590580445355867
46000 0.017055359716886858
47000 0.01752396740618798
48000 0.01799641940269145
49000 0.01847293726081924
50000 0.018953466853321332
51000 0.019437954927048827
52000 0.019926143002946855
53000 0.02041793732764923
54000 0.020913307538099758
55000 0.021412272647498464
57000 0.022421576750639712
58000 0.022931702354724404
60000 0.023962938718321055
61000 0.024484229182972068
63000 0.02553857379364141
66000 0.027149936616520716
67000 0.02

In [37]:
data.describe()

,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
count,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06,1.373228e+06
mean,1.606378e+04,4.140407e+05,1.519224e-03,1.541184e-04,1.034850e-19,1.512979e-03,7.282112e-07,3.665972e-01
std,1.286860e+05,8.133498e+06,1.217041e-02,3.027533e-03,1.236770e-02,1.229366e-02,9.070772e-07,2.104982e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.522092e-03,0.000000e+00,5.035278e-07,2.202330e-01
25%,3.000000e+01,1.450000e+02,2.837235e-06,5.397337e-08,-1.519298e-03,2.777946e-06,5.035278e-07,2.202330e-01
50%,2.340000e+02,1.493000e+03,2.213043e-05,5.557396e-07,-1.500158e-03,2.180286e-05,5.035278e-07,2.202330e-01
75%,2.131000e+03,1.638800e+04,2.015383e-04,6.100108e-06,-1.322541e-03,1.983562e-04,6.273237e-07,5.255937e-01
max,1.057367e+07,2.686510e+09,1.000000e+00,1.000000e+00,1.004501e+00,1.000000e+00,4.541742e-04,1.000000e+00


In [27]:
data.corr()

,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
main_id,1.000000,-0.082811,-0.031470,-0.082811,-0.031470,-0.081653,-0.081653,-0.161269,-0.192620
listeners,-0.082811,1.000000,0.695504,1.000000,0.695504,0.999526,0.999526,0.187253,0.101941
playcount,-0.031470,0.695504,1.000000,0.695504,1.000000,0.717291,0.717291,0.088277,0.045920
listeners_scaled,-0.082811,1.000000,0.695504,1.000000,0.695504,0.999526,0.999526,0.187253,0.101941
playcount_scaled,-0.031470,0.695504,1.000000,0.695504,1.000000,0.717291,0.717291,0.088277,0.045920
popularity,-0.081653,0.999526,0.717291,0.999526,0.717291,1.000000,1.000000,0.185367,0.100822
popularity_scaled,-0.081653,0.999526,0.717291,0.999526,0.717291,1.000000,1.000000,0.185367,0.100822
pagerank,-0.161269,0.187253,0.088277,0.187253,0.088277,0.185367,0.185367,1.000000,0.603592
pagerank_acc,-0.192620,0.101941,0.045920,0.101941,0.045920,0.100822,0.100822,0.603592,1.000000


In [ ]:
r, p_value = pearsonr(data.pagerank, data.popularity_scaled)
print(r)
print(p_value, "significant" if p_value < 0.05 else "not significant")

In [31]:
G_75 = graph.subgraph(data.loc[int(0.99 * len(data)):, "main_id"])
len(G_75)

5128

In [32]:
pg_75 = nx.pagerank(G_75)
data_75 = data.loc[int(0.99 * len(data)):].copy()
len(data_75) == len(pg_75)

True

In [33]:
data_75["pagerank"] = data_75["main_id"].map(pg_75)
data_75["pagerank_acc"] = data_75['pagerank'].cumsum()
data_75 = data_75.reset_index()
data_75.corr()

,index,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
index,1.000000,-0.143435,0.191339,0.101016,0.191339,0.101016,0.189530,0.189530,0.527981,0.984973
main_id,-0.143435,1.000000,-0.173875,-0.035804,-0.173875,-0.035804,-0.169893,-0.169893,-0.164177,-0.154428
listeners,0.191339,-0.173875,1.000000,0.745029,1.000000,0.745029,0.999612,0.999612,0.288749,0.214353
playcount,0.101016,-0.035804,0.745029,1.000000,0.745029,1.000000,0.763319,0.763319,0.152008,0.109411
listeners_scaled,0.191339,-0.173875,1.000000,0.745029,1.000000,0.745029,0.999612,0.999612,0.288749,0.214353
playcount_scaled,0.101016,-0.035804,0.745029,1.000000,0.745029,1.000000,0.763319,0.763319,0.152008,0.109411
popularity,0.189530,-0.169893,0.999612,0.763319,0.999612,0.763319,1.000000,1.000000,0.286001,0.212170
popularity_scaled,0.189530,-0.169893,0.999612,0.763319,0.999612,0.763319,1.000000,1.000000,0.286001,0.212170
pagerank,0.527981,-0.164177,0.288749,0.152008,0.288749,0.152008,0.286001,0.286001,1.000000,0.606187
pagerank_acc,0.984973,-0.154428,0.214353,0.109411,0.214353,0.109411,0.212170,0.212170,0.606187,1.000000


In [34]:
i = 0
current_cum_sum = 0
while i < len(data_75) - 1:
    if data_75.loc[i, "pagerank"] != data_75.loc[i + 1, "pagerank"]:
        current_cum_sum += data_75.loc[i, "pagerank"]
        data_75.loc[i, "pagerank_acc"] = current_cum_sum
        i += 1
    else:
        j = i + 1
        next_cum_sum = current_cum_sum + data_75.loc[i, "pagerank"]
        while data_75.loc[j, "pagerank"] == data_75.loc[i, "pagerank"] and  j < len(data_75) - 1:
            next_cum_sum += data_75.loc[j, "pagerank"]
            j += 1
        mean_cum_sum = (next_cum_sum + current_cum_sum) / 2.0
        for k in range(i, j):
            data_75.loc[k, "pagerank_acc"] = mean_cum_sum
        current_cum_sum = next_cum_sum
        i = j
    if i % 1000 == 0:
        print(i, current_cum_sum)
data_75.loc[i, "pagerank_acc"] = current_cum_sum + data_75.loc[i, "pagerank"]

1000 0.11831447528468579
2000 0.25120729372546075
3000 0.39612685089330874
4000 0.5810899435936205
5000 0.895159192649162


In [35]:
data_75.corr()

,index,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
index,1.000000,-0.143435,0.191339,0.101016,0.191339,0.101016,0.189530,0.189530,0.527981,0.984973
main_id,-0.143435,1.000000,-0.173875,-0.035804,-0.173875,-0.035804,-0.169893,-0.169893,-0.164177,-0.154428
listeners,0.191339,-0.173875,1.000000,0.745029,1.000000,0.745029,0.999612,0.999612,0.288749,0.214353
playcount,0.101016,-0.035804,0.745029,1.000000,0.745029,1.000000,0.763319,0.763319,0.152008,0.109411
listeners_scaled,0.191339,-0.173875,1.000000,0.745029,1.000000,0.745029,0.999612,0.999612,0.288749,0.214353
playcount_scaled,0.101016,-0.035804,0.745029,1.000000,0.745029,1.000000,0.763319,0.763319,0.152008,0.109411
popularity,0.189530,-0.169893,0.999612,0.763319,0.999612,0.763319,1.000000,1.000000,0.286001,0.212170
popularity_scaled,0.189530,-0.169893,0.999612,0.763319,0.999612,0.763319,1.000000,1.000000,0.286001,0.212170
pagerank,0.527981,-0.164177,0.288749,0.152008,0.288749,0.152008,0.286001,0.286001,1.000000,0.606187
pagerank_acc,0.984973,-0.154428,0.214353,0.109411,0.214353,0.109411,0.212170,0.212170,0.606187,1.000000


In [36]:
data_75.tail()

,index,main_id,listeners,playcount,listeners_scaled,playcount_scaled,popularity,popularity_scaled,pagerank,pagerank_acc
5123,512744,255180,2321703,47773305,0.219574,0.017783,0.216819,0.217915,0.001892,0.991779
5124,512745,762646,1403,13224,0.000133,0.000005,-0.002341,0.000131,0.000907,0.992686
5125,512746,177,3979125,68405038,0.376324,0.025462,0.372488,0.372607,0.002759,0.995445
5126,512747,2327,2288231,87452010,0.216408,0.032552,0.216291,0.217391,0.001704,0.997149
5127,512748,491324,75375,8583151,0.007129,0.003195,0.005106,0.007531,0.002851,1.000000


In [48]:
r, p_value = pearsonr(data_75.pagerank, data_75.popularity_scaled)
print(r)
print(p_value, "significant" if p_value < 0.05 else "not significant")

0.2860012925772193
3.9988860098119855e-97 significant


In [55]:
type(p_value)

numpy.float64

In [35]:
driver.close()